# Data Police UK 

This notebook is to briefly demonstrate how to implement the ETL pipeline using downloaded street-level crime and outcomes in City of London in January 2023.

In [1]:
# Load packages
import numpy as np
import pandas as pd

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

The unzip files contain three type of files of which names ends with `street`, `outcomes` and `stop-and-search`.

In [2]:
# read outcomes file
df_outcomes = pd.read_csv('data/2023-01-city-of-london-outcomes.csv')
# read street-level crime file
df_street = pd.read_csv('data/2023-01-city-of-london-street.csv')

In [3]:
# identical columns in street-level crime and outcomes
intersection_col = list(set(df_outcomes.columns).intersection(set(df_street.columns)))
# merge street level and outcome
data = pd.merge(df_street, df_outcomes, on=intersection_col,how='left')

In [4]:

# merge street level crime and outcomes
data = pd.merge(df_street, df_outcomes, on=intersection_col,how='left')

# drop duplicates in Crime ID
data.drop_duplicates(subset=['Crime ID'],inplace=True)

# sort month in descending order
data.sort_values(['Month'], ascending = False, inplace = True)

# if Outcome type is empty, missing_outcome returns yes, else returns no.
data['missing_outcome'] = data['Outcome type'].apply(lambda x : 'yes' if (x == np.NaN) else 'no')

# If missing_outcome is yes,  Authority no outcome returns the values ofReported by, 
# else returns empty string, where Reported by is authorities names.

data['Authority no outcome'] = data.apply(lambda x : x['Reported by'] \
                                          if x['missing_outcome'] == 'yes' else '', axis=1)

# slice `Authority_no_outcome` and crime id
Authority_no_outcome = data[['Crime ID','Authority no outcome']]

# query entries where missing_outcome is "no" to remove entries without outcome
data = data.query('missing_outcome == "no"')

# drop temp column 'missing_outcome' and Authority no outcome with all possibile 
data.drop(['missing_outcome'], axis=1, inplace=True)
data.drop(['Authority no outcome'], axis=1, inplace=True)

# merge Authority_no_outcome to data
data = pd.merge(data,Authority_no_outcome, on=['Crime ID'], how = 'left')
# reset index
data.reset_index(drop=True, inplace=True)

# columns ['Falls within'] is the nearest police station

In [5]:
data.shape

(664, 14)

In [6]:
data.head(2)

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Outcome type,Authority no outcome
0,4a14d4745da0a2219ecf913fdc28a0c84ae8408954cac6...,2023-01,City of London Police,City of London Police,-0.106220,51.518275,On or near B500,E01000916,Camden 027B,Other theft,Under investigation,NaN,NaN,
1,133f556f4f531988a458fc91dfe2ca39a01cfbd9abe530...,2023-01,City of London Police,City of London Police,-0.084754,51.512882,On or near Corbet Court,E01032739,City of London 001F,Theft from the person,Under investigation,NaN,NaN,


In [7]:
# save to local path
data.to_excel('data/CityofLondon_crime_2023_01_street_outcomes.xlsx')